# Explore the OpenBreweryDB API 

In [1]:
# Import dependencies 
import pandas as pd 
import numpy as np
import requests 
import json
from pprint import pprint

In [ ]:
https://sandbox-api.brewerydb.com/v2/
Sandbox Key:c13c821fb670e2fb8cf9a31b4d335d96

In [2]:
brewery_url = "https://sandbox-api.brewerydb.com/v2/"
brewery_data = requests.get(brewery_url).json()
pprint(brewery_data)

{'data': 'You have reached the BreweryDB.com API.  For access, check out '
         'http://www.brewerydb.com/developers',
 'message': 'Request Successful',
 'status': 'success'}


In [ ]:
brewery_url = "https://api.openbrewerydb.org/breweries?per_page=25"
brewery_data = requests.get(brewery_url).json()
pprint(brewery_data)

In [ ]:
brewery_df = pd.DataFrame(brewery_data)
brewery_df

# Exploring the CSV format 

In [3]:
# Read in the CSV using pandas and save into dataframe 
brewery_df = pd.read_csv("https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv")
brewery_df

,id,name,brewery_type,street,city,state,postal_code,website_url,phone,created_at,updated_at,country,longitude,latitude,tags,notes
0,siluria-brewing-company,Siluria Brewing Company,planning,NaN,Alabaster,Alabama,35007-8501,http://www.siluriabrewing.com,2.054828e+09,2018-07-24 01:32:48,2018-08-23 23:20:27,United States,-86.816377,33.244281,NaN,NaN
1,cheaha-brewing-co,Cheaha Brewing Co,brewpub,1208 Walnut Ave,Anniston,Alabama,36201-4526,http://www.cheahabrewingcompany.com,2.567707e+09,2018-07-24 01:32:47,2018-08-23 23:20:09,United States,-85.833774,33.660167,NaN,NaN
2,avondale-brewing-co,Avondale Brewing Co,micro,201 41st St S,Birmingham,Alabama,35222-1932,http://www.avondalebrewing.com,2.057775e+09,2018-07-24 01:32:47,2018-08-23 23:19:58,United States,-86.774322,33.524521,NaN,NaN
3,trim-tab-brewing,Trim Tab Brewing,micro,2721 5th Ave S,Birmingham,Alabama,35233-3401,http://www.trimtabbrewing.com,2.057031e+09,2018-07-24 01:32:48,2018-08-23 23:20:31,United States,-86.791400,33.512849,NaN,NaN
4,cahaba-brewing-co,Cahaba Brewing Co,micro,4500 5th Ave S Ste C,Birmingham,Alabama,35222-2911,http://www.cahababrewing.com,2.059669e+09,2018-07-24 01:32:47,2018-08-11 21:35:44,United States,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7918,sheridan-brewing-company,Sheridan Brewing company,planning,NaN,Sheridan,Wyoming,82801,NaN,NaN,2018-07-24 01:34:38,2018-08-11 21:40:39,United States,NaN,NaN,NaN,NaN
7919,luminous-brewhouse,Luminous Brewhouse,micro,504 Broadway St,Sheridan,Wyoming,82801-3619,http://www.luminousbrewhouse.com,NaN,2018-07-24 01:34:37,2018-08-24 16:46:54,United States,-106.953486,44.802972,NaN,NaN
7920,black-tooth-brewing-co,Black Tooth Brewing Co,micro,312 Broadway St,Sheridan,Wyoming,82801-3917,http://www.blacktoothbrewingcompany.com,3.076752e+09,2018-07-24 01:34:37,2018-08-24 16:46:41,United States,-106.953506,44.800895,NaN,NaN
7921,ten-sleep-brewing-company,Ten Sleep Brewing Company,micro,2549 Hwy 16,Ten Sleep,Wyoming,82442,http://www.tensleepbrewingco.com,3.073662e+09,2018-07-24 01:34:38,2018-08-11 21:40:39,United States,NaN,NaN,NaN,NaN


In [4]:
# Overview of brewery dataframe 
brewery_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7923 entries, 0 to 7922
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7923 non-null   object 
 1   name          7923 non-null   object 
 2   brewery_type  7923 non-null   object 
 3   street        6845 non-null   object 
 4   city          7923 non-null   object 
 5   state         7923 non-null   object 
 6   postal_code   7923 non-null   object 
 7   website_url   6489 non-null   object 
 8   phone         6980 non-null   float64
 9   created_at    7923 non-null   object 
 10  updated_at    7923 non-null   object 
 11  country       7923 non-null   object 
 12  longitude     4737 non-null   float64
 13  latitude      4737 non-null   float64
 14  tags          0 non-null      float64
 15  notes         0 non-null      float64
dtypes: float64(5), object(11)
memory usage: 990.5+ KB


In [5]:
# Identify and count null values in each column in df 
nan_count = brewery_df.isnull().sum()
nan_count[nan_count > 0].sort_values(ascending = False)

notes          7923
tags           7923
latitude       3186
longitude      3186
website_url    1434
street         1078
phone           943
dtype: int64

In [ ]:
# Drop the columns that contain only null values (notes and tags)
